In [16]:
import pandas as pd 

file_path = '/Users/lenoxherman/Tippers/tipDataSet/tip.csv'
# Extract the CSV file from the zip archive

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)


In [17]:
df


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def prepare_data(df):
  #removing columns that are not needed
  # df.drop('smoker', axis=1, inplace=True)
  day_mapping = {'Sun': 1, 'Sat': 1, 'Fri': 1, 'Thur':0}
  # Create a new column 'day_encoded' based on Weekday or Weekend
  df['day_encoded'] = df['day'].map(day_mapping)
  # Creates a new column 'percent_tip' based on the percentage of the tip amount
  df['percent_tip'] = (df['tip'] / df['total_bill']) * 100
  # Round the 'percent_tip' column to two decimal places
  df['percent_tip'] = df['percent_tip'].round()
  #makes groups for tips to 25+
  tip_groups = [0, 10, 15, 20, 25, float('inf')]
  label_tip_groups = ['0-10%', '10-15%', '15-20%', '20-25%', '25+']
  df['tip_groups'] = pd.cut(df['percent_tip'], bins = tip_groups, labels= label_tip_groups)
  df['tip_groups'] = df['tip_groups'].astype(str)
  df['tip_groups'] = df['tip_groups'].str.extract(r'(\d+)').astype(float)

  le.fit(df["tip_groups"])
  #turns the labels in 'tip' to a number
  y = le.transform(df["tip_groups"])
  #print(y)
  #removing 'tip' col bc now held by y
  df = df.drop(["tip_groups"], axis = 1)
  #converted into “one-hot encoded” 0-1 columns
  df = pd.get_dummies(df)
  return df, y
X_train, y_train = prepare_data(df)

In [19]:
df

,total_bill,tip,sex,smoker,day,time,size,day_encoded,percent_tip,tip_groups
0,16.99,1.01,Female,No,Sun,Dinner,2,1,6.0,0.0
1,10.34,1.66,Male,No,Sun,Dinner,3,1,16.0,15.0
2,21.01,3.50,Male,No,Sun,Dinner,3,1,17.0,15.0
3,23.68,3.31,Male,No,Sun,Dinner,2,1,14.0,10.0
4,24.59,3.61,Female,No,Sun,Dinner,4,1,15.0,10.0
...,...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,1,20.0,15.0
240,27.18,2.00,Female,Yes,Sat,Dinner,2,1,7.0,0.0
241,22.67,2.00,Male,Yes,Sat,Dinner,2,1,9.0,0.0
242,17.82,1.75,Male,No,Sat,Dinner,2,1,10.0,0.0


## Visualizations

In [20]:
print(y_train)


[0 2 2 1 1 2 3 1 1 3 2 1 0 2 2 2 2 3 3 2 3 1 1 2 2 1 1 2 2 1 1 1 2 1 2 1 1
 2 1 2 1 1 3 1 2 2 3 2 0 2 2 3 1 2 2 2 0 0 2 1 2 1 2 3 1 2 1 4 0 1 2 2 1 2
 1 1 2 1 1 2 1 2 2 1 1 1 1 3 3 1 0 2 2 4 1 1 1 1 1 1 3 2 0 2 2 1 2 2 3 4 3
 1 1 1 2 2 2 1 1 1 2 1 2 1 2 1 2 1 2 0 0 1 1 2 2 1 2 1 1 3 2 2 1 2 1 2 0 1
 2 4 2 1 2 0 0 2 0 1 2 1 2 2 1 1 2 1 1 1 1 2 2 2 4 0 3 0 1 1 4 0 1 3 0 4 0
 3 2 0 2 2 0 3 0 1 3 2 2 1 1 1 3 2 1 1 2 2 1 0 0 2 0 2 2 2 3 0 1 1 2 0 2 4
 3 2 1 1 2 1 2 1 0 2 4 1 2 1 0 0 1 2 0 0 0 2]


In [23]:
from itertools import combinations
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

all_quan_cols = ["total_bill", "tip", "size", "percent_tip", "size", "tip_groups", "day_encoded"]
all_qual_cols= ["sex", "smoker","day", "time"]

predictor_cols = "tip_groups"
target_col = "tip"

LR = LogisticRegression()
result = LR.fit(X_train[predictor_cols], y_train)
LR.fit(X_train[predictor_cols], y_train)
LR.score(X_train[predictor_cols], y_train)
LR.coef_

KeyError: 'tip_groups'